At first, we need to load the Python module.

In [1]:
from tecan import Fluent, DiTi, VariableContainer

Next, we need to connect to the SiLA2 Server that controls FluentControl. 
**Please start the SiLA2 Server before you run the next line.**
Further, you may have to adapt the IP address, in case you are not running the SiLA2 Server locally.

In [2]:
f = Fluent('192.168.178.23', 50052)

2021-03-24 08:47:18,440 Overwriting given hostname "localhost" with IP address "192.168.178.23" since IP takes precedence.
2021-03-24 08:47:18,629 Starting SiLA2 service client for service SilaFluentController with service name: SilaFluentControllerClient
2021-03-24 08:47:18,632 successfully connected to the server


#### Inspecting the API
You can inspect the methods available using the standard `dir` operator

In [3]:
dir(f)

['_Fluent__FORMAT',
 '_Fluent__client',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_labware',
 'aspirate',
 'close_method',
 'discover',
 'dispense',
 'drop_tips',
 'finish_execution',
 'get_all_runnable_methods',
 'get_fingers',
 'get_tips',
 'get_variable_names',
 'get_variable_value',
 'pause_run',
 'prepare_method',
 'remove_labware',
 'resume_run',
 'run_method',
 'set_location',
 'set_variable_value',
 'shutdown',
 'start_fluent',
 'stop_method',
 'transfer_labware',
 'transfer_labware_back_to_base',
 'user_prompt']

#### Starting FluentControl and starting methods
Using start_fluent(), you can start FluentControl or connect with a running instance

In [4]:
f.start_fluent(simulation_mode=True)

2021-03-24 08:47:52,232 starting Fluent...
2021-03-24 08:47:52,372 started


As soon as the connection is established, you can connect to it and list available methods or prepare a method for execution

In [5]:
print(f.get_all_runnable_methods())

['demo']


In [6]:
f.close_method()

2021-03-24 08:48:16,849 method closed


For the remainder of this demo, you will need to have a method that opens an API channel. As an example, you can use the method from [here](https://gitlab.com/tecan/fluent-sila2-connector/uploads/de9771548ee81e77c64c1210a5cab1d7/demo_method.zeia)

In [7]:
f.prepare_method("demo")

2021-03-24 08:48:49,718 preparing method...
2021-03-24 08:48:49,878 method ready to run


Once you prepared a method, it is possible to list variable names and run the method.

In [8]:
variable_names = f.get_variable_names()
print(variable_names)

['not_a_test', 'test']


In [9]:
f.run_method()

2021-03-24 08:49:19,196 method running


#### Commands
As soon as a method opens an API channel, it is possible to change the state of variables or issue commands.
First, we demonstrate this by changing a variable value.

In [10]:
value = f.get_variable_value('test')
print(value)

test


In [11]:
f.set_variable_value('test', "new test")

2021-03-24 08:50:23,862 Value successfully set


We double-check that the variable value has indeed changed.

In [12]:
value = f.get_variable_value('test')
print(value)

new test


Next, lets add some labware and perform some liquid transfer operations.

In [13]:
f.add_labware("48 Well Flat[001]", "48 Well Flat", "Nest61mm_Pos", 1)

2021-03-24 08:50:57,155 Labware added successfully


You can use the standard `inspect` module to review the parameters and there is inline documentation available.

In [15]:
import inspect
inspect.signature(f.add_labware)

<Signature (labware_name: str, labware_type: str, target_location: str, position=0, rotation=0, has_lid=False, barcode='')>

Next, we transfer this labware to a different location

In [16]:
f.transfer_labware("48 Well Flat[001]", "Nest61mm_Pos", 2)

2021-03-24 08:52:14,921 transferring labware
2021-03-24 08:52:18,693 labware successfully transferred


For disposable tips, there are collections of constants available, so you don't have to memorize the tip names.

In [18]:
f.get_tips(10, 70, DiTi.FCA_200_UL_FILTERED_SBS)

2021-03-24 08:52:49,201 executing...
2021-03-24 08:52:56,266 done


Of course, you can also issue commands to aspirate or dispense liquids directly.

In [19]:
f.aspirate(42, "100ml_3", "Water Free Single", 0)

2021-03-24 08:53:28,318 aspirating...
2021-03-24 08:53:37,533 finished aspirating


In [20]:
f.dispense(23, "96 Well Round[002]", "Water Free Single", 0)

2021-03-24 08:53:45,332 dispensing...
2021-03-24 08:53:48,031 finished dispensing


Finally, don't forget to drop the tips.

In [21]:
f.drop_tips("FCA Thru Deck Waste Chute_1")

2021-03-24 08:53:58,318 dropping tips...
2021-03-24 08:54:02,169 dropped


#### Cleanup
At the end, you can cleanly finish execution.

In [22]:
f.finish_execution()

2021-03-24 08:54:21,661 successfully finished execution.


In [23]:
f.shutdown(1)